In [1]:
### CODE TỪ MAMMO-CLIP

import os  # Thư viện thao tác với hệ điều hành (tạo, đọc thư mục, tệp)
import dicomsdl  # Thư viện để xử lý dữ liệu DICOM
import numpy as np  # Thư viện cho tính toán số học
import cv2  # OpenCV, thư viện xử lý ảnh
from tqdm.auto import tqdm  # Thư viện tạo thanh tiến trình
from datetime import datetime
# Đường dẫn tới thư mục chứa ảnh DICOM và ảnh PNG
input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/images"
output_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG-MAMMO-CLIP-CROPPED-TEXT_REMOVE"

# Hàm tìm chỉ số của các vùng không liên tục trong mảng nhị phân
def np_CountUpContinuingOnes(b_arr):
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)

    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]

    return right - left - 1  # Trả về khoảng cách giữa các vùng không liên tục

# Hàm để trích xuất vùng có chứa ngực từ ảnh
def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 40, 0, img)  # Loại bỏ các vùng nền tối (giá trị <= 40)
    height, _ = img.shape

    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]

    img = img[:, col_ind]

    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]

    return img_copy[row_ind][:, col_ind]

# Hàm lưu ảnh sau khi xử lý vào thư mục đầu ra, không thay đổi kích thước
def save_imgs(in_path, out_path, SIZE=(912, 1520)):
    dicom = dicomsdl.open(in_path)  # Mở tệp DICOM
    data = dicom.pixelData()  # Lấy dữ liệu pixel từ tệp DICOM
    data = data[5:-5, 5:-5]  # Cắt bớt các viền ảnh

    if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
        data = np.amax(data) - data

    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)

    img = ExtractBreast(data)  # Trích xuất vùng chứa ngực
    img = cv2.resize(img, SIZE, interpolation=cv2.INTER_AREA)

    cv2.imwrite(out_path, img)  # Lưu ảnh dưới dạng PNG tại `out_path`

# Lấy danh sách tất cả các tệp DICOM trong thư mục đầu vào
dicom_files = [f for f in os.listdir(input_folder) if f.endswith('.dicom')]

# Tạo thư mục để lưu ảnh PNG nếu chưa có
os.makedirs(output_folder, exist_ok=True)

# Vòng lặp xử lý tất cả các tệp DICOM
for dicom_file in tqdm(dicom_files, total=len(dicom_files)):
    in_path = os.path.join(input_folder, dicom_file)  # Đường dẫn tới tệp DICOM
    out_path = os.path.join(output_folder, dicom_file.replace('.dicom', '.png'))  # Đổi đuôi thành PNG

    try:
        save_imgs(in_path, out_path)  # Gọi hàm xử lý và lưu ảnh
    except Exception as e:
        print(f"Không thể xử lý {in_path}: {e}")  # Báo lỗi nếu không xử lý được ảnh




current_time = datetime.now().strftime("%Y%m%d_%H%M%S")  # Định dạng thời gian
completed_folder_name = f"da_xong_{current_time}"
completed_folder_path = os.path.join("/media/mountHDD2/chuyenmt/BrEaST/Mammo/", completed_folder_name)

# Tạo thư mục
os.makedirs(completed_folder_path, exist_ok=True)

print(f"Đã tạo thư mục: {completed_folder_path}")

  0%|          | 0/20000 [00:00<?, ?it/s]

Đã tạo thư mục: /media/mountHDD2/chuyenmt/BrEaST/Mammo/da_xong_20241026_012401


In [2]:
## CODE TỪ MAMMO-CLIP
# import os  # Thư viện thao tác với hệ điều hành (tạo, đọc thư mục, tệp)
# import dicomsdl  # Thư viện để xử lý dữ liệu DICOM
# import numpy as np  # Thư viện cho tính toán số học
# import cv2  # OpenCV, thư viện xử lý ảnh
# from tqdm.auto import tqdm  # Thư viện tạo thanh tiến trình

# # Đường dẫn tới thư mục chứa ảnh DICOM và ảnh PNG
# input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/images"
# output_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG-2"

# # Hàm tìm chỉ số của các vùng không liên tục trong mảng nhị phân
# def np_CountUpContinuingOnes(b_arr):
#     left = np.arange(len(b_arr))
#     left[b_arr > 0] = 0
#     left = np.maximum.accumulate(left)

#     rev_arr = b_arr[::-1]
#     right = np.arange(len(rev_arr))
#     right[rev_arr > 0] = 0
#     right = np.maximum.accumulate(right)
#     right = len(rev_arr) - 1 - right[::-1]

#     return right - left - 1  # Trả về khoảng cách giữa các vùng không liên tục

# # Hàm để trích xuất vùng có chứa ngực từ ảnh
# def ExtractBreast(img):
#     img_copy = img.copy()
#     img = np.where(img <= 40, 0, img)  # Loại bỏ các vùng nền tối (giá trị <= 40)
#     height, _ = img.shape

#     y_a = height // 2 + int(height * 0.4)
#     y_b = height // 2 - int(height * 0.4)
#     b_arr = img[y_b:y_a].std(axis=0) != 0
#     continuing_ones = np_CountUpContinuingOnes(b_arr)
#     col_ind = np.where(continuing_ones == continuing_ones.max())[0]

#     img = img[:, col_ind]

#     _, width = img.shape
#     x_a = width // 2 + int(width * 0.4)
#     x_b = width // 2 - int(width * 0.4)
#     b_arr = img[:, x_b:x_a].std(axis=1) != 0
#     continuing_ones = np_CountUpContinuingOnes(b_arr)
#     row_ind = np.where(continuing_ones == continuing_ones.max())[0]

#     return img_copy[row_ind][:, col_ind]

# # Hàm lưu ảnh sau khi xử lý vào thư mục đầu ra, không thay đổi kích thước
# def save_imgs(in_path, out_path, SIZE=(912, 1520)):
#     dicom = dicomsdl.open(in_path)  # Mở tệp DICOM
#     data = dicom.pixelData()  # Lấy dữ liệu pixel từ tệp DICOM
#     data = data[5:-5, 5:-5]  # Cắt bớt các viền ảnh

#     if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
#         data = np.amax(data) - data

#     data = data - np.min(data)
#     data = data / np.max(data)
#     data = (data * 255).astype(np.uint8)

#     img = ExtractBreast(data)  # Trích xuất vùng chứa ngực
#     img = cv2.resize(img, SIZE, interpolation=cv2.INTER_AREA)

#     cv2.imwrite(out_path, img)  # Lưu ảnh dưới dạng PNG tại `out_path`

# # Lấy danh sách tất cả các tệp DICOM trong thư mục đầu vào
# dicom_files = [f for f in os.listdir(input_folder) if f.endswith('.dicom')][:20]  # Chỉ lấy 20 tệp đầu tiên

# # Tạo thư mục để lưu ảnh PNG nếu chưa có
# os.makedirs(output_folder, exist_ok=True)

# # Vòng lặp xử lý tất cả các tệp DICOM
# for dicom_file in tqdm(dicom_files, total=len(dicom_files)):
#     in_path = os.path.join(input_folder, dicom_file)  # Đường dẫn tới tệp DICOM
#     out_path = os.path.join(output_folder, dicom_file.replace('.dicom', '.png'))  # Đổi đuôi thành PNG

#     try:
#         save_imgs(in_path, out_path)  # Gọi hàm xử lý và lưu ảnh
#     except Exception as e:
#         print(f"Không thể xử lý {in_path}: {e}")  # Báo lỗi nếu không xử lý được ảnh


In [3]:
# import matplotlib.pyplot as plt
# import cv2
# from pathlib import Path

# # Đường dẫn đến file ảnh
# # img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG/ffff43ffbe56fd959c81d83a5ecdc4dc.png'
# img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG-2/6c178a7dd55215b2707e083e7bd1bb8a.png'

# # Kiểm tra xem file có tồn tại không
# if Path(img_path).is_file():
#     # Đọc file ảnh
#     image = cv2.imread(img_path)

#     # Chuyển từ BGR (mặc định của OpenCV) sang RGB để hiển thị đúng
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     print(image_rgb.shape)

#     # Hiển thị ảnh
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image_rgb)
#     plt.axis('off')  # Ẩn trục
#     plt.show()
# else:
#     print(f"File không tồn tại: {img_path}")


In [4]:
# import matplotlib.pyplot as plt
# import cv2
# from pathlib import Path

# # Đường dẫn đến file ảnh
# # img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG/ffff43ffbe56fd959c81d83a5ecdc4dc.png'
# img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG-2/e3ba831628c057a62568e89e8dfdc233.png'

# # Kiểm tra xem file có tồn tại không
# if Path(img_path).is_file():
#     # Đọc file ảnh
#     image = cv2.imread(img_path)

#     # Chuyển từ BGR (mặc định của OpenCV) sang RGB để hiển thị đúng
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     print(image_rgb.shape)

#     # Hiển thị ảnh
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image_rgb)
#     plt.axis('off')  # Ẩn trục
#     plt.show()
# else:
#     print(f"File không tồn tại: {img_path}")


In [5]:
# import os  # Thư viện thao tác với hệ điều hành (tạo, đọc thư mục, tệp)
# import dicomsdl  # Thư viện để xử lý dữ liệu DICOM
# import numpy as np  # Thư viện cho tính toán số học
# import cv2  # OpenCV, thư viện xử lý ảnh
# from tqdm.auto import tqdm  # Thư viện tạo thanh tiến trình

# # Đường dẫn tới thư mục chứa ảnh DICOM và ảnh PNG
# input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/images"
# output_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG-2"

# # Hàm tìm chỉ số của các vùng không liên tục trong mảng nhị phân
# def np_CountUpContinuingOnes(b_arr):
#     left = np.arange(len(b_arr))
#     left[b_arr > 0] = 0
#     left = np.maximum.accumulate(left)

#     rev_arr = b_arr[::-1]
#     right = np.arange(len(rev_arr))
#     right[rev_arr > 0] = 0
#     right = np.maximum.accumulate(right)
#     right = len(rev_arr) - 1 - right[::-1]

#     return right - left - 1  # Trả về khoảng cách giữa các vùng không liên tục

# # Hàm để trích xuất vùng có chứa ngực từ ảnh
# def ExtractBreast(img):
#     img_copy = img.copy()
#     img = np.where(img <= 40, 0, img)  # Loại bỏ các vùng nền tối (giá trị <= 40)
#     height, _ = img.shape

#     y_a = height // 2 + int(height * 0.4)
#     y_b = height // 2 - int(height * 0.4)
#     b_arr = img[y_b:y_a].std(axis=0) != 0
#     continuing_ones = np_CountUpContinuingOnes(b_arr)
#     col_ind = np.where(continuing_ones == continuing_ones.max())[0]

#     img = img[:, col_ind]

#     _, width = img.shape
#     x_a = width // 2 + int(width * 0.4)
#     x_b = width // 2 - int(width * 0.4)
#     b_arr = img[:, x_b:x_a].std(axis=1) != 0
#     continuing_ones = np_CountUpContinuingOnes(b_arr)
#     row_ind = np.where(continuing_ones == continuing_ones.max())[0]

#     return img_copy[row_ind][:, col_ind]

# # Hàm lưu ảnh sau khi xử lý vào thư mục đầu ra, không thay đổi kích thước
# def save_imgs(in_path, out_path, laterality='L'):
#     dicom = dicomsdl.open(in_path)  # Mở tệp DICOM
#     data = dicom.pixelData()  # Lấy dữ liệu pixel từ tệp DICOM
#     data = data[5:-5, 5:-5]  # Cắt bớt các viền ảnh

#     if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
#         data = np.amax(data) - data

#     data = data - np.min(data)
#     data = data / np.max(data)
#     data = (data * 255).astype(np.uint8)

#     img = ExtractBreast(data)  # Trích xuất vùng chứa ngực

#     if laterality == 'R':
#         img = cv2.flip(img, 1)  # Lật ảnh theo trục ngang

#     cv2.imwrite(out_path, img)  # Lưu ảnh dưới dạng PNG tại `out_path`

# # Lấy danh sách tất cả các tệp DICOM trong thư mục đầu vào
# dicom_files = [f for f in os.listdir(input_folder) if f.endswith('.dicom')]

# # Tạo thư mục để lưu ảnh PNG nếu chưa có
# os.makedirs(output_folder, exist_ok=True)

# # Vòng lặp xử lý tất cả các tệp DICOM
# for dicom_file in tqdm(dicom_files, total=len(dicom_files)):
#     in_path = os.path.join(input_folder, dicom_file)  # Đường dẫn tới tệp DICOM
#     out_path = os.path.join(output_folder, dicom_file.replace('.dicom', '.png'))  # Đổi đuôi thành PNG

#     try:
#         save_imgs(in_path, out_path)  # Gọi hàm xử lý và lưu ảnh
#     except Exception as e:
#         print(f"Không thể xử lý {in_path}: {e}")  # Báo lỗi nếu không xử lý được ảnh


In [6]:
# import pydicom
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# from tqdm import tqdm
# from datetime import datetime

# # Đường dẫn tới thư mục chứa file DICOM
# input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/images"
# # Đường dẫn tới thư mục lưu ảnh PNG
# output_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG"

# # Tạo thư mục đầu ra nếu chưa tồn tại
# os.makedirs(output_folder, exist_ok=True)

# # Duyệt qua tất cả file trong thư mục đầu vào
# for filename in tqdm(os.listdir(input_folder)):
#     if filename.endswith(".dicom"):  # Kiểm tra xem file có đuôi .dicom không
#         dicom_file_path = os.path.join(input_folder, filename)

#         # Đọc ảnh DICOM
#         dicom_image = pydicom.dcmread(dicom_file_path)

#         # Chuyển dữ liệu pixel của ảnh DICOM sang mảng numpy
#         image_array = dicom_image.pixel_array

#         # Đảm bảo chuyển đổi sang uint8 để hiển thị đúng
#         if image_array.dtype != np.uint8:
#             image_array = (image_array / image_array.max() * 255).astype(np.uint8)

#         # Kiểm tra Photometric Interpretation
#         if dicom_image.PhotometricInterpretation == "MONOCHROME1":
#             # Đảo màu: từ đen nền trắng sang trắng nền đen
#             image_array = 255 - image_array

#         # Lưu ảnh PNG từ DICOM
#         output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.png")
#         plt.imsave(output_path, image_array, cmap='gray', vmin=0, vmax=255)

#         # print(f"Đã lưu ảnh: {output_path}")

# print("Hoàn thành chuyển đổi tất cả ảnh DICOM sang PNG.")

# current_time = datetime.now().strftime("%Y%m%d_%H%M%S")  # Định dạng thời gian
# completed_folder_name = f"da_xong_{current_time}"
# completed_folder_path = os.path.join("/media/mountHDD2/chuyenmt/BrEaST/Mammo/", completed_folder_name)

# # Tạo thư mục
# os.makedirs(completed_folder_path, exist_ok=True)

# print(f"Đã tạo thư mục: {completed_folder_path}")


In [7]:
import numpy as np
import cv2
import os
from joblib import Parallel, delayed
from pathlib import Path
import dicomsdl

# Đường dẫn đến thư mục chứa tệp DICOM và thư mục đầu ra
input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/images"
output_folder = f"/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG_2"

# Tạo thư mục đầu ra nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

def dicom_file_to_ary(path):
    dcm_file = dicomsdl.open(str(path))
    data = dcm_file.pixelData()

    # Chuẩn hóa dữ liệu pixel
    data = (data - data.min()) / (data.max() - data.min())

    # Kiểm tra Photometric Interpretation và đảo màu nếu cần
    if dcm_file.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
        data = 1 - data

    # Chuyển đổi về kiểu uint8
    data = (data * 255).astype(np.uint8)
    return data

def process_file(file_path):
    if file_path.suffix.lower() in ['.dcm', '.dicom']:  # Kiểm tra định dạng file
        processed_ary = dicom_file_to_ary(file_path)
        
        # Lưu ảnh PNG
        output_path = os.path.join(output_folder, f"{file_path.stem}.png")
        cv2.imwrite(output_path, processed_ary)
        print(f"Đã lưu: {output_path}")  # Thông báo khi lưu thành công

# Lấy danh sách tất cả các tệp DICOM trong thư mục
input_directory = Path(input_folder)
all_dicom_files = list(input_directory.glob("*.dcm")) + list(input_directory.glob("*.dicom"))

# Chỉ xử lý 30 tệp đầu tiên
images_to_process = all_dicom_files[:30]

# Sử dụng Parallel để xử lý các tệp song song
with Parallel(n_jobs=os.cpu_count()) as p:
    p(delayed(process_file)(file) for file in images_to_process)

print("Hoàn thành chuyển đổi 30 ảnh DICOM đầu tiên sang PNG.")


Hoàn thành chuyển đổi 30 ảnh DICOM đầu tiên sang PNG.


In [8]:
# import pydicom
# from pathlib import Path
# import numpy as np
# from PIL import Image
# import multiprocessing as mp

# # Đường dẫn tới thư mục chứa file DICOM
# input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG"

# # Thư mục lưu trữ ảnh PNG đã xử lý
# output_folder = "train_images_processed"
# !rm -rf {output_folder}  # Xóa thư mục cũ nếu có
# !mkdir -p {output_folder}  # Tạo thư mục mới

# # Hàm chuyển đổi DICOM thành mảng numpy
# def dicom_file_to_ary(path):
#     dicom = pydicom.read_file(path)
#     data = dicom.pixel_array

#     if dicom.PhotometricInterpretation == "MONOCHROME1":
#         data = np.amax(data) - data  # Đảo màu cho ảnh âm bản

#     data = data - np.min(data)  # Chuẩn hóa về [0, 1]
#     data = data / np.max(data)
#     data = (data * 255).astype(np.uint8)  # Chuyển thành 8-bit

#     return data

# # Hàm xử lý từng file DICOM và lưu dưới dạng PNG
# def process_dicom(image_path):
#     try:
#         processed_ary = dicom_file_to_ary(image_path)  # Chuyển đổi DICOM thành numpy array
#         im = Image.fromarray(processed_ary)  # Không resize, giữ nguyên kích thước gốc
#         im.save(f"{output_folder}/{image_path.stem}.png")  # Lưu ảnh PNG
#         print(f"Processed: {image_path}")
#     except Exception as e:
#         print(f"Error processing {image_path}: {e}")

# # Lấy danh sách tất cả các file DICOM trong thư mục đầu vào
# dicom_files = list(Path(input_folder).glob("*.dcm"))  # Tìm tất cả các file .dcm

# # Xử lý song song các file DICOM
# with mp.Pool(64) as p:
#     p.map(process_dicom, dicom_files)


In [9]:
# import pydicom
# from pathlib import Path
# import numpy as np
# from PIL import Image

# # Đường dẫn tới thư mục chứa file DICOM
# input_folder = "/media/mountHDD3/data_storage/biomedical_data/Dataset/images"

# # Thư mục lưu trữ ảnh PNG đã xử lý
# output_folder =  "/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG"
 

# # Hàm chuyển đổi DICOM thành mảng numpy
# def dicom_file_to_ary(path):
#     dicom = pydicom.read_file(path)
#     data = dicom.pixel_array

#     if dicom.PhotometricInterpretation == "MONOCHROME1":
#         data = np.amax(data) - data  # Đảo màu cho ảnh âm bản

#     data = data - np.min(data)  # Chuẩn hóa về [0, 1]
#     data = data / np.max(data)
#     data = (data * 255).astype(np.uint8)  # Chuyển thành 8-bit

#     return data

# # Lấy danh sách tất cả các file DICOM trong thư mục đầu vào
# dicom_files = list(Path(input_folder).glob("*.dicom"))

# # Chuyển đổi ảnh đầu tiên trong danh sách
# if dicom_files:
#     first_image = dicom_files[0]  # Lấy ảnh đầu tiên
#     try:
#         processed_ary = dicom_file_to_ary(first_image)  # Chuyển đổi DICOM thành numpy array
#         im = Image.fromarray(processed_ary)  # Không resize, giữ nguyên kích thước gốc
#         im.save(f"{output_folder}/{first_image.stem}.png")  # Lưu ảnh PNG
#         print(f"Processed: {first_image}")
#     except Exception as e:
#         print(f"Error processing {first_image}: {e}")
# else:
#     print("No DICOM files found.")


In [10]:
# import matplotlib.pyplot as plt
# import cv2
# from pathlib import Path

# # Đường dẫn đến file ảnh
# img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG/ffff43ffbe56fd959c81d83a5ecdc4dc.png'

# # Kiểm tra xem file có tồn tại không
# if Path(img_path).is_file():
#     # Đọc file ảnh
#     image = cv2.imread(img_path)

#     # Chuyển từ BGR (mặc định của OpenCV) sang RGB để hiển thị đúng
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Hiển thị ảnh
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image_rgb)
#     plt.axis('off')  # Ẩn trục
#     plt.show()
# else:
#     print(f"File không tồn tại: {img_path}")


In [11]:
# image_rgb.shape

In [12]:
# import matplotlib.pyplot as plt
# import cv2
# from pathlib import Path

# # Đường dẫn đến file ảnh
# img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo/Processed/vindr_png/ffff43ffbe56fd959c81d83a5ecdc4dc.png'

# # Kiểm tra xem file có tồn tại không
# if Path(img_path).is_file():
#     # Đọc file ảnh
#     image = cv2.imread(img_path)

#     # Chuyển từ BGR (mặc định của OpenCV) sang RGB để hiển thị đúng
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Hiển thị ảnh
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image_rgb)
#     plt.axis('off')  # Ẩn trục
#     plt.show()
# else:
#     print(f"File không tồn tại: {img_path}")


In [13]:
# image_rgb.shape

In [14]:
# import matplotlib.pyplot as plt
# import cv2
# from pathlib import Path

# # Đường dẫn đến file ảnh
# img_path = '/media/mountHDD3/data_storage/biomedical_data/Dataset/VinDr-Mammo-PNG/ffa132203a8e326ecfaee102566c68dd.png'

# # Kiểm tra xem file có tồn tại không
# if Path(img_path).is_file():
#     # Đọc file ảnh
#     image = cv2.imread(img_path)

#     # Chuyển từ BGR (mặc định của OpenCV) sang RGB để hiển thị đúng
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Hiển thị ảnh
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image_rgb)
#     plt.axis('off')  # Ẩn trục
#     plt.show()
# else:
#     print(f"File không tồn tại: {img_path}")
